In [209]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys
import pyaudio
import scipy
from scipy import signal
import sympy
from sympy import symbols, fraction, simplify, degree
from sympy.functions import arg
z = symbols('z')

import os
import struct
import time
from tkinter import TclError

# to display the output in a window outside the jupyter notebook
%matplotlib tk

# defining constants 
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [154]:
from sympy.utilities.lambdify import lambdify
def evalHz(Hz,w):
    func = lambdify(z,Hz,'numpy')
    return func(np.exp(1j*w))


In [220]:
def recordAudio(a,b):
    WAVE_OUTPUT_FILENAME = 'input.wav'
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = secs

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Audio recorded!")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    #sleep for a few secs?

    play = input("Want to listen to your raw input? (Y/N)\n")

    if play == 'Y' or play == 'y':
        #play audio code here

    #sleep
    print("Here's a plot of your raw input!")
    spf = wave.open(WAVE_OUTPUT_FILENAME, 'r')

    #Extract Raw Audio from Wav File
    signal = spf.readframes(-1)
    signal = np.fromstring(signal, 'Int16')
    filtered = scipy.signal.lfilter(a,b,signal)

    plt.figure(1)
    plt.title('Signal Wave...')
    plt.plot(filtered)


IndentationError: expected an indented block (<ipython-input-220-e62854a3cd5d>, line 46)

In [167]:
#LCCDE coeffs
def HzLCCDE():
    print('I am assuming that this system is LTI\n')
    a_k = []
    b_m = []
    Nz = 0
    Dz = 0
    ylen = int(input('Enter lenght of y[k] (starting with y[n] and going up to the last non-zero coeff of y[k]): '))
    for i in range(0, ylen):
        p = int(input())
        a_k.append(p) #append the ith coeff of y[k] to the list of coeffs of y[k]
        Dz += p*z**(-1*i) 
    
    xlen = int(input('Enter lenght of x[m] (starting with x[n] and going up to the last non-zero coeff of x[m]): '))
    for i in range(0, xlen):
        p = int(input())
        b_m.append(p) #append the ith coeff of x[m] to thr list of coeffs of x[m]
        Nz += p*z**(-1*i)
        
    Hz = Nz/Dz
    return Hz
    

In [168]:
#poles-zeros of H(z)
def HzPoleZero():
    zeros = []
    poles = []
    Hz = 1
    a = int(input('Enter number of zeros: '))
    for i in range(0, a):
        p = complex(input())
        Hz *= (z-p)
        zeros.append(p) #append the ith zero to the list of zeros

    b = int(input('Enter number of poles: '))
    for j in range(0, b):
        k = complex(input())
        Hz /= (z-k)
        poles.append(k) #append the jth pole to the list of poles
    return Hz


In [169]:
#impulse response
def HzImpulseResponse():
    N = int(input('What is the length of your impulse response h[n]? : '))
    h_n = []
    Hz = 0
    print('Please enter the values of h[t], starting from 0 and going to N-1:\n')
    for i in range(0, N):
        p = int(input())
        h_n.append(p) #append the ith impulse to the list of impulses
        Hz += p*z**(-1*i)
    return Hz
    

In [187]:
#ask for name, to personalize the user experience
user_name = input('Please enter your name:\n')
print('Hey',user_name,'!')
print('Thanks for using my audio filtering platform!')
print("Let's start by designing our filter\n")
#ask the user what type of input they want to use
option = input("Would you like to design your own filter using,\nA. LCCDE coefficients?\nB. poles and zeros of transfer function H(z)?\nC. impulse response h[n]?\n")

Please enter your name:
Shivangi
Hey Shivangi !
Thanks for using my audio filtering platform!
Let's start by designing our filter

Would you like to design your own filter using,
A. LCCDE coefficients?
B. poles and zeros of transfer function H(z)?
C. impulse response h[n]?
A


In [219]:
#main block for constructing the filter
Hz = 1
if option == 'A' or option == 'a':
    Hz = HzLCCDE()
elif option == 'B' or option == 'b':
    Hz = HzPoleZero()
elif option == 'C' or option == 'c':
    Hz = HzImpulseResponse()
else:
    option = input('Enter a valid option (A/B/C)')
print('Here is the transfer function of your filter:')
simplify(Hz)
print(Hz)
Nz,Dz = fraction(Hz)
a = []
b = []
for i in range (0,2*CHUNK):
    k1 = Nz.coeff(z,-1*i)
    a.append(k1)
    k2 = Dz.coeff(z,-1*i)
    b.append(k2)
print('Run the next cell to start audio processing')

I am assuming that this system is LTI

Enter lenght of y[k] (starting with y[n] and going up to the last non-zero coeff of y[k]): 4
1
2
1
2
Enter lenght of x[m] (starting with x[n] and going up to the last non-zero coeff of x[m]): 5
1
2
1
3
4
Here is the transfer function of your filter:
(1 + 2/z + z**(-2) + 3/z**3 + 4/z**4)/(1 + 2/z + z**(-2) + 2/z**3)
Run the next cell to start audio processing


In [225]:
#code for saving audio for offline analysis
save_rep = input('Do you want to save your audio in a new file (could be useful for offline processing? (Y/N)\n')
if save_rep == 'Y' or save_rep == 'y':
    filename = input('What do you want the audio file name to be? {format: <entername>.wav}\n ')
else:
    filename = "input.wav"
secs = int(input("How long do you want the input to be (in seconds)?\n")) 
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = secs
WAVE_OUTPUT_FILENAME = filename
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("start recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)  

print("Audio recorded!")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

spf = wave.open(WAVE_OUTPUT_FILENAME, 'r')

#Extract Raw Audio from Wav File
signal = spf.readframes(-1)
signal = np.fromstring(signal, 'Int16')
filtered = scipy.signal.lfilter(a,b,signal,axis = -1, zi = None)
print(filtered)

Do you want to save your audio in a new file (could be useful for offline processing? (Y/N)
N
How long do you want the input to be (in seconds)?
5
start recording
Audio recorded!


/home/randomshit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
/home/randomshit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


KeyboardInterrupt: 

In [ ]:
# code for real-time audio analysis
# lacks the filtering component

fig, ax = plt.subplots(1, figsize=(15, 7))

p = pyaudio.PyAudio()
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x_n = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x_n, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break